In [124]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [125]:
df = pd.read_csv('/content/cleanedData2.csv')
df.head()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6735 entries, 0 to 6734
Data columns (total 30 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   resultId                 6735 non-null   int64  
 1   raceId                   6735 non-null   int64  
 2   driverId                 6735 non-null   int64  
 3   constructorId_x          6735 non-null   int64  
 4   grid                     6735 non-null   int64  
 5   positionText             6735 non-null   object 
 6   positionOrder            6735 non-null   int64  
 7   statusId                 6735 non-null   int64  
 8   year                     6735 non-null   int64  
 9   round                    6735 non-null   int64  
 10  circuitId                6735 non-null   int64  
 11  driverRef                6735 non-null   object 
 12  nationality_x            6735 non-null   object 
 13  constructorRef           6735 non-null   object 
 14  status                  

In [126]:
df.head()

,resultId,raceId,driverId,constructorId_x,grid,positionText,positionOrder,statusId,year,round,...,driverErrorRate,driverScore,pos_clean,momentum_3,momentum_5,momentum_10,constructor_avg_pos,constructor_momentum,driver_circuit_avg,constructor_circuit_avg
0,20323,337,4,6,3,1,1,1,2010,1,...,0.010753,82.045162,1.0,-1.0,-1.0,-1.0,6.631579,-1.0,9.333333,7.3125
1,20324,337,13,6,2,2,2,1,2010,1,...,0.025806,81.005493,2.0,-2.0,-2.0,-2.0,6.631579,-1.5,8.142857,7.3125
2,20325,337,1,1,4,3,3,1,2010,1,...,0.006270,91.445403,3.0,-3.0,-3.0,-3.0,6.394737,-3.0,3.266667,10.6250
3,20326,337,20,9,1,4,4,1,2010,1,...,0.015564,86.171946,4.0,-4.0,-4.0,-4.0,6.026316,-4.0,7.083333,7.1875
4,20327,337,3,131,5,5,5,1,2010,1,...,0.000000,83.025085,5.0,-5.0,-5.0,-5.0,9.157895,-5.0,4.166667,4.3125


In [127]:
# --- feature sets ---
WITH_QUALI = [
    "grid","quali_best_s_filled",
    "momentum_3","momentum_5","momentum_10",
    "driverErrorRate","driverScore",
    "constructor_avg_pos","constructor_momentum",
    "driver_circuit_avg","constructor_circuit_avg",
    "driverRef","constructorId_x","circuitId"
]

WITHOUT_QUALI = [
    "momentum_3","momentum_5","momentum_10",
    "driverErrorRate","driverScore",
    "constructor_avg_pos","constructor_momentum",
    "driver_circuit_avg","constructor_circuit_avg",
    "driverRef","constructorId_x","circuitId"
]

In [128]:
lineup = [
    {"driverRef": "verstappen", "constructorRef": "red_bull"},
    {"driverRef": "tsunoda",      "constructorRef": "red_bull"},
    {"driverRef": "leclerc",    "constructorRef": "ferrari"},
    {"driverRef": "hamilton",      "constructorRef": "ferrari"},
    {"driverRef": "norris",     "constructorRef": "mclaren"},
    {"driverRef": "piastri",    "constructorRef": "mclaren"},
    {"driverRef": "antonelli",   "constructorRef": "mercedes"},
    {"driverRef": "russell",    "constructorRef": "mercedes"},
    {"driverRef": "alonso",     "constructorRef": "aston_martin"},
    {"driverRef": "stroll",     "constructorRef": "aston_martin"},
    {"driverRef": "colapinto",       "constructorRef": "alpine"},
    {"driverRef": "gasly",      "constructorRef": "alpine"},
    {"driverRef": "albon",      "constructorRef": "williams"},
    {"driverRef": "sainz",   "constructorRef": "williams"},
    {"driverRef": "hulkenberg",  "constructorRef": "sauber"},      # formerly Alfa Romeo → Stake F1
    {"driverRef": "bortoleto",   "constructorRef": "sauber"},
    {"driverRef": "hadjar",    "constructorRef": "rb"},         # formerly AlphaTauri → RB
    {"driverRef": "lawson",  "constructorRef": "rb"},
    {"driverRef": "ocon", "constructorRef": "haas"},
    {"driverRef": "bearman", "constructorRef": "haas"}
]

In [129]:
quali_dict = {
    "verstappen": {"grid": 1, "quali_best_s_filled": 78.792},
    "norris":     {"grid": 2, "quali_best_s_filled": 78.869},
    "piastri":    {"grid": 3, "quali_best_s_filled": 78.982},
    "leclerc":    {"grid": 4, "quali_best_s_filled": 79.007},
    "hamilton":   {"grid": 5, "quali_best_s_filled": 79.124},
    "russell":    {"grid": 6, "quali_best_s_filled": 79.157},
    "antonelli":  {"grid": 7, "quali_best_s_filled": 79.200},
    "bortoleto":  {"grid": 8, "quali_best_s_filled": 79.390},
    "alonso":     {"grid": 9, "quali_best_s_filled": 79.424},
    "tsunoda":    {"grid": 10, "quali_best_s_filled": 79.519},
    "bearman":    {"grid": 11, "quali_best_s_filled": 79.446},
    "hulkenberg": {"grid": 12, "quali_best_s_filled": 79.498},
    "sainz":      {"grid": 13, "quali_best_s_filled": 79.528},
    "albon":      {"grid": 14, "quali_best_s_filled": 79.583},
    "ocon":       {"grid": 15, "quali_best_s_filled": 79.707},
    "hadjar":     {"grid": 16, "quali_best_s_filled": 79.917},
    "stroll":     {"grid": 17, "quali_best_s_filled": 79.948},
    "colapinto":  {"grid": 18, "quali_best_s_filled": 79.992},
    "gasly":      {"grid": 19, "quali_best_s_filled": 80.103},
    "lawson":     {"grid": 20, "quali_best_s_filled": 80.279},
}


In [130]:
# function to create the two models with quali data and without quali data
cutoff = 14

def train2Model(df):
    y = df["positionOrder"]

    Xq = pd.get_dummies(df[WITH_QUALI], columns=["driverRef","constructorId_x","circuitId"], drop_first=True)
    Xn = pd.get_dummies(df[WITHOUT_QUALI], columns=["driverRef","constructorId_x","circuitId"], drop_first=True)

    # Extract the index from the filtered DataFrame
    train_idx = df[(df["year"] == 2025) & (df["round"] <= cutoff)].index

    modelQ = XGBRegressor(n_estimators=500, learning_rate=0.01, random_state=42)
    modelQ.fit(Xq.loc[train_idx], y.loc[train_idx])

    modelN = XGBRegressor(n_estimators=500, learning_rate=0.01, random_state=42)
    modelN.fit(Xn.loc[train_idx], y.loc[train_idx])

    return {
        "with_quali": {"model": modelQ, "columns": Xq.columns},
        "no_quali":   {"model": modelN, "columns": Xn.columns},
    }

artifacts = train2Model(df)

In [131]:
cutoff = 7

def train2Model(df):
    y = df["positionOrder"]

    Xq = pd.get_dummies(df[WITH_QUALI], columns=["driverRef","constructorId_x","circuitId"], drop_first=True)
    Xn = pd.get_dummies(df[WITHOUT_QUALI], columns=["driverRef","constructorId_x","circuitId"], drop_first=True)

    train_idx = df[(df["year"] == 2025) & (df["round"] <= cutoff)].index

    modelQ = XGBRegressor(n_estimators=500, learning_rate=0.01, random_state=42)
    modelQ.fit(Xq.loc[train_idx], y.loc[train_idx])

    modelN = XGBRegressor(n_estimators=500, learning_rate=0.01, random_state=42)
    modelN.fit(Xn.loc[train_idx], y.loc[train_idx])

    return {
        "with_quali": {"model": modelQ, "columns": Xq.columns},
        "no_quali":   {"model": modelN, "columns": Xn.columns},
    }

artifacts = train2Model(df)


In [132]:
def build_snapshot_for_lineup(df, lineup, upcoming_year, upcoming_round, upcoming_circuitId):
    lineup_df = pd.DataFrame(lineup)

    # Historical strictly before this GP
    hist = df[(df["year"] < upcoming_year) |
              ((df["year"] == upcoming_year) & (df["round"] < upcoming_round))]

    # Last known row per driver
    last_idx = hist.groupby("driverRef")["raceId"].idxmax()
    last = hist.loc[last_idx].copy()

    # Merge lineup onto last known state
    snap = lineup_df.merge(last, on="driverRef", how="left", suffixes=("","_hist"))

    # Overwrite context
    snap["year"] = upcoming_year
    snap["round"] = upcoming_round
    snap["circuitId"] = upcoming_circuitId

    # If you use constructorId_x in training, enforce it here
    if "constructorId_x" not in snap.columns:
        snap["constructorId_x"] = snap["constructorRef"]

    # … then continue with the fill logic (grid, quali_best_s_filled, features etc.)
    return snap


In [133]:
def predict_pre_quali(snapshot_df, artifacts):
    use = snapshot_df[[c for c in WITHOUT_QUALI if c in snapshot_df.columns]]
    X = pd.get_dummies(use, columns=["driverRef","constructorId_x","circuitId"], drop_first=True)
    X = X.reindex(columns=artifacts["no_quali"]["columns"], fill_value=0)
    preds = artifacts["no_quali"]["model"].predict(X)
    out = snapshot_df[["driverRef"]].copy()
    out["predicted_position"] = preds
    out = out.sort_values("predicted_position").reset_index(drop=True)
    out["proj_rank"] = out.index + 1
    return out

def predict_post_quali(snapshot_df, artifacts, quali_dict):
    dfq = snapshot_df.copy()
    def qget(d, k): return (quali_dict.get(d, {}) or {}).get(k, np.nan)
    dfq["grid"] = dfq["driverRef"].map(lambda d: qget(d, "grid"))
    if "quali_best_s_filled" in dfq.columns:
        dfq["quali_best_s_filled"] = dfq["driverRef"].map(lambda d: qget(d, "quali_best_s_filled"))
    if dfq["quali_best_s_filled"].isna().any():
        worst = dfq["quali_best_s_filled"].max(skipna=True)
        dfq["quali_best_s_filled"] = dfq["quali_best_s_filled"].fillna((worst + 10.0) if pd.notna(worst) else 100.0)
    if dfq["grid"].isna().any():
        dfq["grid"] = dfq["grid"].fillna(10)

    use = dfq[[c for c in WITH_QUALI if c in dfq.columns]]
    X = pd.get_dummies(use, columns=["driverRef","constructorId_x","circuitId"], drop_first=True)
    X = X.reindex(columns=artifacts["with_quali"]["columns"], fill_value=0)
    preds = artifacts["with_quali"]["model"].predict(X)
    out = dfq[["driverRef","grid"]].copy()
    out["predicted_position"] = preds
    out = out.sort_values("predicted_position").reset_index(drop=True)
    out["proj_rank"] = out.index + 1
    return out


In [134]:
UPCOMING_YEAR, UPCOMING_ROUND, UPCOMING_CID = 2025, 16, 14  # example

snap = build_snapshot_for_lineup(
    df, lineup,
    upcoming_year=UPCOMING_YEAR,
    upcoming_round=UPCOMING_ROUND,
    upcoming_circuitId=UPCOMING_CID
)

# BEFORE quali
pre = predict_post_quali(snap, artifacts, quali_dict)
print(pre.head(20))

     driverRef  grid  predicted_position  proj_rank
0      piastri     3            1.203461          1
1   verstappen     1            3.055792          2
2       norris     2            3.326544          3
3      leclerc     4            3.705675          4
4      russell     6            4.620312          5
5       stroll    17            8.651054          6
6        albon    14            8.808834          7
7       hadjar    16            8.826685          8
8      bearman    11           10.508388          9
9   hulkenberg    12           11.105417         10
10      lawson    20           11.313128         11
11    hamilton     5           11.749248         12
12       sainz    13           11.758142         13
13      alonso     9           12.508499         14
14   colapinto    18           12.806907         15
15        ocon    15           13.630243         16
16       gasly    19           13.855806         17
17   bortoleto     8           14.370871         18
18   antonel

In [135]:
col = snap.loc[snap.driverRef == "stroll"]
print(col.T)   # transpose so you see all features vertically


                                    9
driverRef                      stroll
constructorRef           aston_martin
resultId                      27050.0
raceId                         1159.0
driverId                        840.0
constructorId_x                 117.0
grid                             19.0
positionText                        7
positionOrder                     7.0
statusId                          1.0
year                             2025
round                              16
circuitId                          14
nationality_x                Canadian
constructorRef_hist      aston_martin
status                       Finished
qualifyId                     10851.0
quali_best_s                      NaN
no_quali_time                     1.0
quali_best_s_filled           1012.64
accident_flag                     0.0
driverErrorRate              0.016484
driverScore                 71.746805
pos_clean                         7.0
momentum_3                  -9.333333
momentum_5  

In [136]:
for feat in ["momentum_3","momentum_5","momentum_10",
             "driverScore","driver_circuit_avg",
             "constructor_avg_pos","constructor_momentum"]:
    print(feat, " -> ",
          col[feat].values[0],
          " (median=", df[feat].median(), ")")

momentum_3  ->  -9.333333333333334  (median= -10.0 )
momentum_5  ->  -9.8  (median= -10.0 )
momentum_10  ->  -12.8  (median= -10.0 )
driverScore  ->  71.74680523096812  (median= 77.37278960664021 )
driver_circuit_avg  ->  10.6  (median= 10.727272727272728 )
constructor_avg_pos  ->  12.344827586206897  (median= 11.770833333333334 )
constructor_momentum  ->  -10.0  (median= -11.4 )


In [137]:
print("colopinto in df?", "verstappen" in df["driverRef"].unique())

colopinto in df? False


In [138]:
# 0) the 20 you intend to predict for
intended = {d["driverRef"] for d in lineup}

# 1) confirm the snapshot only includes your 20
print("snapshot drivers:", set(snap["driverRef"]))
missing = intended - set(snap["driverRef"])
extra   = set(snap["driverRef"]) - intended
print("MISSING:", missing)
print("EXTRA:", extra)

# if EXTRA not empty, restrict:
snap = snap[snap["driverRef"].isin(intended)].copy()
assert len(snap)==20, f"Expected 20, got {len(snap)}"

# 2) check NaNs on critical features
crit = ["momentum_3","momentum_5","momentum_10","driverScore",
        "constructor_avg_pos","constructor_momentum",
        "driver_circuit_avg","constructor_circuit_avg"]
print(snap[crit].isna().sum().sort_values(ascending=False))

# 3) check team/category values match training
print("unique constructorId_x in snapshot:", snap["constructorId_x"].unique())
# Compare with what the model saw in training:
print("example of trained columns containing constructor dummies:",
      [c for c in artifacts["no_quali"]["columns"] if "constructorId_x_" in c][:10])

# 4) quick peek at the rows that looked wrong
print(snap.query("driverRef in ['colopinto','hadjar','bortoleto','lawson','antonelli']")
         [["driverRef","constructorId_x","momentum_3","driverScore",
           "constructor_avg_pos","constructor_momentum","driver_circuit_avg","constructor_circuit_avg"]])


snapshot drivers: {'leclerc', 'colapinto', 'tsunoda', 'alonso', 'ocon', 'antonelli', 'hamilton', 'piastri', 'hadjar', 'hulkenberg', 'albon', 'sainz', 'lawson', 'bearman', 'bortoleto', 'russell', 'stroll', 'verstappen', 'norris', 'gasly'}
MISSING: set()
EXTRA: set()
momentum_3                 1
momentum_5                 1
momentum_10                1
driverScore                1
constructor_avg_pos        1
constructor_momentum       1
driver_circuit_avg         1
constructor_circuit_avg    1
dtype: int64
unique constructorId_x in snapshot: [ nan   9.   6.   1. 131. 117. 214.   3.  15. 215. 210.]
example of trained columns containing constructor dummies: ['constructorId_x_3', 'constructorId_x_4', 'constructorId_x_5', 'constructorId_x_6', 'constructorId_x_9', 'constructorId_x_10', 'constructorId_x_15', 'constructorId_x_51', 'constructorId_x_117', 'constructorId_x_131']
    driverRef  constructorId_x  momentum_3  driverScore  constructor_avg_pos  \
6   antonelli            131.0  -14.000

In [139]:
import shap
X_new = pd.get_dummies(
    snap[[c for c in ["momentum_3","momentum_5","momentum_10",
                      "driverErrorRate","driverScore",
                      "constructor_avg_pos","constructor_momentum",
                      "driver_circuit_avg","constructor_circuit_avg",
                      "driverRef","constructorId_x","circuitId"] if c in snap.columns]],
    columns=["driverRef","constructorId_x","circuitId"], drop_first=True
).reindex(columns=artifacts["no_quali"]["columns"], fill_value=0)

model = artifacts["no_quali"]["model"]  # or use with_quali model if after quali
explainer = shap.TreeExplainer(model)
sv = explainer.shap_values(X_new)

# Top global drivers of the snapshot predictions:
abs_mean = pd.Series(np.abs(sv).mean(axis=0), index=X_new.columns).sort_values(ascending=False)
print(abs_mean.head(25))

# Per-driver top pushes/pulls:
per_driver = pd.DataFrame(sv, columns=X_new.columns, index=snap["driverRef"])
print(per_driver.loc["verstappen"].sort_values(ascending=False).head(10))  # what pushed his prediction DOWN/UP


constructor_momentum       1.783011
driver_circuit_avg         1.665583
momentum_3                 0.992829
momentum_5                 0.511824
constructor_circuit_avg    0.305073
driverRef_alonso           0.191853
driverScore                0.170296
momentum_10                0.124771
driverErrorRate            0.079626
driverRef_albon            0.033847
circuitId_22               0.028663
constructor_avg_pos        0.026498
circuitId_77               0.018785
driverRef_sainz            0.013456
driverRef_lawson           0.010267
circuitId_21               0.010003
circuitId_3                0.009463
driverRef_gasly            0.008508
circuitId_17               0.007317
driverRef_ocon             0.006211
driverRef_colapinto        0.005349
constructorId_x_6          0.004621
constructorId_x_117        0.004573
constructorId_x_131        0.004130
driverRef_norris           0.003945
dtype: float32
driver_circuit_avg     1.759234
momentum_10            0.163151
driverRef_albon      

In [140]:
print(snap.loc[snap.driverRef=="verstappen", ["momentum_3","momentum_5","momentum_10","driver_circuit_avg","driverScore"]])


   momentum_3  momentum_5  momentum_10  driver_circuit_avg  driverScore
0         NaN         NaN          NaN                 NaN          NaN
